<a href="https://colab.research.google.com/github/Upenbhatt/Seismology/blob/main/PGA_using_event_id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install usgs-libcomcat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=77f304cdfc1825ffe9fe3ffddaedf396378315f3606d9c0170307cbded1c3695
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [1]:
pip install geopy

In [2]:
from datetime import datetime
import json
import os
from glob import glob
import pandas as pd

from libcomcat.search import search,get_event_by_id
from geopy.distance import distance

In [8]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client


In [9]:
client = Client("USGS")


In [ ]:
## TO FETCH EVENT ID

In [24]:
starttime=datetime(2021,1,1,0,0)
endtime=datetime(2023,8,22,0,0)
minlatitude=24.40
maxlatitude=25.67
minlongitude=89.95
maxlongitude=93.09
eventtype='earthquake'
events=search(starttime=starttime,
              endtime=endtime,
              minlatitude=minlatitude,
              maxlatitude=maxlatitude,
              minlongitude=minlongitude,
              maxlongitude=maxlongitude,
              eventtype=eventtype,
              producttype='shaKEMAP')
print(events)

[us6000kzvl 2023-08-14 14:49:48.477000 (24.926,92.228) 35.1 km M5.3]


In [25]:
print('Got',len(events),'earthquake events.')
events[0]

Got 1 earthquake events.


us6000kzvl 2023-08-14 14:49:48.477000 (24.926,92.228) 35.1 km M5.3

In [26]:
events=[e for e in events if e.hasProduct('dyfi')]
print('Got',len(events),'events with ShakeMap and DYFI products.')

Got 1 events with ShakeMap and DYFI products.


In [31]:
exampleEvent=events[0]
exampleEvent

us6000kzvl 2023-08-14 14:49:48.477000 (24.926,92.228) 35.1 km M5.3

In [32]:
eventdir='./events'
os.makedirs(eventdir,exist_ok=True)

columns=['id','lat','lon','val','nresp']

allevents={ 'shakemap':{} }

shakemapParam='pga'
def getShakeMap(e):

    # Check if loaded previously
    if 'shakemap' in allevents and e.id in allevents['shakemap']:
        return allevents['shakemap'][e.id]

    datafile='stationlist.json'
    propTag=shakemapParam

    # Check if already saved to file
    savefile='%s/%s/%s' % (eventdir,e.id,datafile)
    fileglob=glob(savefile)
    if not fileglob:
        eventdata=get_event_by_id(e.id)
        product=eventdata.getProducts('shakemap')[0]
        os.makedirs('%s/%s' % (eventdir,e.id),exist_ok=True)
        url=product.getContent(datafile,savefile)
        print('Downloaded',url)

    with open(savefile,'r') as f:
        stations=json.load(f)['features']

    nresp=1
    results=[]
    for s in stations:
        val=s['properties'][propTag]
        if not val or val=='null':
            continue

        # Don't include macroseismic stations!
        id=s['id']
        if id[0:4]=='DYFI' or id[0:3]=='OBS':
            continue

        lon,lat=s['geometry']['coordinates']
        results.append([id,
                        round(float(lat),4),
                        round(float(lon),4),
                        float(val),
                        float(nresp)])

    results=pd.DataFrame(results,columns=columns)
    allevents['shakemap'][e.id]=results
    return results


shakemap=getShakeMap(exampleEvent)
print('This event has',len(shakemap),'ShakeMap stations.')
shakemap.head(5)

Downloaded https://earthquake.usgs.gov/product/shakemap/us6000kzvl/us/1692111141841/download/stationlist.json
This event has 0 ShakeMap stations.


Empty DataFrame
Columns: [id, lat, lon, val, nresp]
Index: []

In [33]:
dyfiType='1km'
minresp=3

allevents['dyfi']={}
def getDyfi(e):

    # Check if loaded previously
    if 'dyfi' in allevents and e.id in allevents['dyfi']:
        return allevents['dyfi'][e.id]

    datafile='dyfi_geo_%s.geojson' % dyfiType
    propTag='cdi'

    # Check if already saved to file
    savefile='%s/%s/%s' % (eventdir,e.id,datafile)
    fileglob=glob(savefile)
    if not fileglob:
        eventdata=get_event_by_id(e.id)
        product=eventdata.getProducts('dyfi')[0]
        os.makedirs('%s/%s' % (eventdir,e.id),exist_ok=True)
        url=product.getContent(datafile,savefile)
        print('Downloaded',url)

    with open(savefile,'r') as f:
        stations=json.load(f)['features']

    results=[]
    for s in stations:
        val=s['properties'][propTag]
        if not val or val=='null':
            continue
        nresp=s['properties']['nresp']
        if nresp<minresp:
            continue
        # Clean up UTM name
        id=s['properties']['name'].split('<br>')[0]

        # Get centroid of UTM cell
        lat=0; lon=0
        for pt in s['geometry']['coordinates'][0]:
            lat+=float(pt[0])
            lon+=float(pt[1])
        ncorners=len(s['geometry']['coordinates'][0])
        lat,lon=(round(lon/ncorners,4),round(lat/ncorners,4))

        results.append([id,lat,lon,val,nresp])

    results=pd.DataFrame(results,columns=columns)
    allevents['dyfi'][e.id]=results
    return results


dyfi=getDyfi(exampleEvent)
print('This event has',len(dyfi),'DYFI locations.')
dyfi[0:5]

Downloaded https://earthquake.usgs.gov/product/dyfi/us6000kzvl/us/1692538577756/dyfi_geo_1km.geojson
This event has 2 DYFI locations.


id      lat      lon  val  nresp
0  UTM:(46Q 0231 2634 1000)  23.7993  90.3649  3.7      3
1  UTM:(46Q 0236 2632 1000)  23.7821  90.4143  4.0      3

In [38]:
pip install git+https://code.usgs.gov/ghsc/esi/shakemap-sampling-tool.git

  Cloning https://code.usgs.gov/ghsc/esi/shakemap-sampling-tool.git to /tmp/pip-req-build-f0lcqhns
  Running command git clone --filter=blob:none --quiet https://code.usgs.gov/ghsc/esi/shakemap-sampling-tool.git /tmp/pip-req-build-f0lcqhns
  Resolved https://code.usgs.gov/ghsc/esi/shakemap-sampling-tool.git to commit e51822c8f9da838917e387f290ef285417ab74ec
  Preparing metadata (setup.py) ... done
  Created wheel for shakemap-sampling-tool: filename=shakemap_sampling_tool-0.0.0-py3-none-any.whl size=10089 sha256=2885f41812e0f550399ce7ec5bfde3f9a7078eabf7e7dd113344685c1c932d96
  Stored in directory: /tmp/pip-ephem-wheel-cache-zw9fhy_b/wheels/a0/62/c4/11c3da0ac002d86d59a49afbcb1b9c030f577b7d2741ec4372
Successfully built shakemap-sampling-tool


In [39]:
pip install --upgrade git+https://code.usgs.gov/ghsc/esi/shakemap-sampling-tool.git

  Cloning https://code.usgs.gov/ghsc/esi/shakemap-sampling-tool.git to /tmp/pip-req-build-yq3yqcq5
  Running command git clone --filter=blob:none --quiet https://code.usgs.gov/ghsc/esi/shakemap-sampling-tool.git /tmp/pip-req-build-yq3yqcq5
  fatal: unable to access 'https://code.usgs.gov/ghsc/esi/shakemap-sampling-tool.git/': Received HTTP/0.9 when not allowed
  error: RPC failed; curl 1 Received HTTP/0.9 when not allowed
  fatal: error reading section header 'shallow-info'
  fatal: unable to access 'https://code.usgs.gov/ghsc/esi/shakemap-sampling-tool.git/': Received HTTP/0.9 when not allowed
  Resolved https://code.usgs.gov/ghsc/esi/shakemap-sampling-tool.git to commit e51822c8f9da838917e387f290ef285417ab74ec
  Preparing metadata (setup.py) ... done


In [43]:
!sst sample us6000kzvl -c "Sonapur_Tunnel" 25.106 92.361

            id      lat      lon sample_lat sample_lon sample_distance_m       vs30  MMI_C  MMI_Sigma_HH_YD  MMI_mean  MMI_phi  MMI_sta_phi  MMI_std  MMI_tau  PGA_mean  PGA_phi  PGA_std  PGA_tau  PGV_mean  PGV_phi  PGV_std  PGV_tau  SA(0.3)_mean  SA(0.3)_phi  SA(0.3)_std  SA(0.3)_tau  SA(1.0)_mean  SA(1.0)_phi  SA(1.0)_std  SA(1.0)_tau  SA(3.0)_mean  SA(3.0)_phi  SA(3.0)_std  SA(3.0)_tau
Sonapur_Tunnel 25.10600 92.36100   25.10833   92.35833          373.3776 626.682861    NaN              NaN  4.486801 1.147391          NaN 1.291067 0.591902  -2.98232 0.584089 0.686706 0.361118  0.600602 0.570795 0.654863 0.320995     -2.686092      0.63903     0.729311     0.351476     -4.526419     0.627339     0.732911     0.378952      -6.83973     0.597926     0.708476      0.38003


In [67]:
!sst sample us6000jpaj -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event us6000jpaj. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [69]:
!sst sample us7000ilc3 -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event us7000ilc3. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [73]:
!sst sample usp000j24a -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event usp000j24a. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [74]:
!sst sample usp000hs8t -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event usp000hs8t. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [75]:
!sst sample usp000gr8t -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event usp000gr8t. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [76]:
!sst sample usp000b8eq -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event usp000b8eq. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [77]:
!sst sample usp000b05s -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event usp000b05s. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [78]:
!sst sample usp00081x0 -c "Sonapur_Tunnel" 25.106 92.361

            id      lat      lon sample_lat sample_lon sample_distance_m       vs30  MMI_mean  MMI_std  PGA_mean  PGA_std  PGV_mean  PGV_std  SA(0.3)_mean  SA(0.3)_std  SA(1.0)_mean  SA(1.0)_std  SA(3.0)_mean  SA(3.0)_std
Sonapur_Tunnel 25.10600 92.36100   25.10833   92.35833          373.3776 626.682861  5.451882 1.242253 -1.905182 0.658486   1.80022 0.737421     -2.079543     0.783057     -3.430394     0.800866     -5.272695     0.781683


In [79]:
!sst sample usp0006g38 -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event usp0006g38. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [80]:
!sst sample usp0005c5t -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event usp0005c5t. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [81]:
!sst sample usp00053fk -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event usp00053fk. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [82]:
!sst sample usp000453c -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event usp000453c. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [83]:
!sst sample usp0002yy2 -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event usp0002yy2. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [84]:
!sst sample usp0002xyr -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event usp0002xyr. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [85]:
!sst sample us7000kg5v -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event us7000kg5v. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [86]:
!sst sample us7000k8uw -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event us7000k8uw. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [89]:
!sst sample us6000k2fw -c "Sonapur_Tunnel" 25.106 92.361

No ShakeMap for event us6000k2fw. Exiting.
Traceback (most recent call last):
  File "/usr/local/bin/sst", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/sampling/bin/sst.py", line 45, in main
    args.func(args)
  File "/usr/local/lib/python3.10/dist-packages/sampling/handlers.py", line 96, in get_sample_handler
    hdf_url = get_hdf_url(args.eventid)
  File "/usr/local/lib/python3.10/dist-packages/sampling/hdf_sampling.py", line 76, in get_hdf_url
    shakemaps = products["shakemap"]
KeyError: 'shakemap'


In [90]:
!sst sample usp0003ct7 -c "Sonapur_Tunnel" 25.106 92.361

            id      lat      lon sample_lat sample_lon sample_distance_m       vs30  MMI_mean  MMI_std  PGA_mean  PGA_std  PGV_mean  PGV_std  SA(0.3)_mean  SA(0.3)_std  SA(1.0)_mean  SA(1.0)_std  SA(3.0)_mean  SA(3.0)_std
Sonapur_Tunnel 25.10600 92.36100   25.10833   92.35833          373.3776 626.682861  3.913188 0.733811 -3.230355 0.588701  0.566894  0.56186     -3.543625     0.720731     -4.702293     0.751788     -6.421623     0.739975


In [91]:
!sst sample usp00081x0 -c "Sonapur_Tunnel" 25.106 92.361

            id      lat      lon sample_lat sample_lon sample_distance_m       vs30  MMI_mean  MMI_std  PGA_mean  PGA_std  PGV_mean  PGV_std  SA(0.3)_mean  SA(0.3)_std  SA(1.0)_mean  SA(1.0)_std  SA(3.0)_mean  SA(3.0)_std
Sonapur_Tunnel 25.10600 92.36100   25.10833   92.35833          373.3776 626.682861  5.451882 1.242253 -1.905182 0.658486   1.80022 0.737421     -2.079543     0.783057     -3.430394     0.800866     -5.272695     0.781683


In [92]:
!sst sample usp0003tx3 -c "Sonapur_Tunnel" 25.106 92.361

            id      lat      lon sample_lat sample_lon sample_distance_m       vs30  MMI_mean  MMI_std  PGA_mean  PGA_std  PGV_mean  PGV_std  SA(0.3)_mean  SA(0.3)_std  SA(1.0)_mean  SA(1.0)_std  SA(3.0)_mean  SA(3.0)_std
Sonapur_Tunnel 25.10600 92.36100   25.10833   92.35833          373.3776 626.682861  3.543582 0.808144 -4.367669 0.692805 -0.585201 0.777121      -3.84336     0.732363     -5.300317      0.74848     -7.309787     0.730917


In [93]:
!sst sample usp000dabz -c "Sonapur_Tunnel" 25.106 92.361

            id      lat      lon sample_lat sample_lon sample_distance_m       vs30  MMI_mean  MMI_std  PGA_mean  PGA_std  PGV_mean  PGV_std  SA(0.3)_mean  SA(0.3)_std  SA(1.0)_mean  SA(1.0)_std  SA(3.0)_mean  SA(3.0)_std
Sonapur_Tunnel 25.10600 92.36100   25.10833   92.35833          373.3776 626.682861  3.842154 0.801835 -3.710235 0.670715 -0.016417 0.773517     -3.261825     0.720895     -4.773751     0.727508     -6.770054     0.714624


In [94]:
!sst sample usp000dg6k -c "Sonapur_Tunnel" 25.106 92.361

            id      lat      lon sample_lat sample_lon sample_distance_m       vs30  MMI_mean  MMI_std  PGA_mean  PGA_std  PGV_mean  PGV_std  SA(0.3)_mean  SA(0.3)_std  SA(1.0)_mean  SA(1.0)_std  SA(3.0)_mean  SA(3.0)_std
Sonapur_Tunnel 25.10600 92.36100   25.10833   92.35833          373.3776 626.682861  3.177922  0.82187 -4.997508 0.727565 -1.482236 0.815327     -4.539772     0.756568     -6.196615     0.753615     -8.427374     0.731932


In [95]:
!sst sample usp0002aJx -c "Sonapur_Tunnel" 25.106 92.361

            id      lat      lon sample_lat sample_lon sample_distance_m       vs30  MMI_mean  MMI_std  PGA_mean  PGA_std  PGV_mean  PGV_std  SA(0.3)_mean  SA(0.3)_std  SA(1.0)_mean  SA(1.0)_std  SA(3.0)_mean  SA(3.0)_std
Sonapur_Tunnel 25.10600 92.36100   25.10833   92.35833          373.3776 626.682861   3.86275 0.795141 -3.743171 0.658764  0.392502  0.70784     -3.073446     0.703547     -4.301209     0.741513     -6.056285     0.732881
